In [26]:
import networkx as nx
from fa2 import ForceAtlas2  #the package should be installed before
import matplotlib.pyplot as plt
from datetime import datetime
from statistics import mean, median, quantiles
from networkx.algorithms.traversal.breadth_first_search import descendants_at_distance
import time
import requests
import glob
import pickle
import os
import shutil
import json
import time
import pandas as pd

In [27]:
df = pd.read_csv("bubble.csv")

In [28]:
names = df["name"].unique()

In [29]:
autPupNets = glob.glob("aG_enrich_monthly_puppies/total/*")
comPupNets = glob.glob("cG_enrich_monthly_puppies/total/*")
autPolNets = glob.glob("aG_enrich_monthly_PoliticalDiscussion/total/*")
comPolNets = glob.glob("cG_enrich_monthly_PoliticalDiscussion/total/*")

In [30]:
def createData(nodes, links):
    data = {}
    data["nodes"]=createNodeList(autGraph, comGraph)
    data["links"]=createLinkList(autGraph)
    return data

In [31]:
def createLinkList(autGraph):
    links = []
    for edge in autGraph.edges():
        link = {}
        link["source"] = edge[0]
        link["target"] = edge[1]
        links.append(link)
    return links

In [32]:
def createNode(autGraph, comGraph, name):
    comments_body = [comGraph.nodes()[comNode]["body"] for comNode in comGraph if comGraph.nodes()[comNode]["author"]==name]
    comments_id = [comNode for comNode in comGraph if comGraph.nodes()[comNode]["author"]==name]
    node = {}
    if(name == comGraph.nodes()[list(comGraph.nodes())[0]]["author"]):
        node["first_author"] = 1
    else: 
        node["first_author"] = 0
    node["name"] = name
    node["id"] = name
    node["created"] =  str(datetime.utcfromtimestamp(int(autGraph.nodes()[name]["created"])))
    node["comments_body"] = comments_body
    node["comments_id"] = comments_id
    return node

In [33]:
def createNodeList(autGraph, comGraph):
    nodes = []
    for name in autGraph.nodes():
        nodes.append(createNode(autGraph, comGraph, name))
    return nodes
        

In [34]:
jsonData = {}
for aNet in autPupNets:
    aNetId = aNet.split("_")[-2]
    if aNetId in names:
        autGraph = nx.read_gpickle(aNet)
        comIdNet = [string for string in comPupNets if aNetId in string]
        comGraph = nx.read_gpickle(comIdNet[0])
        jsonData["t3_"+aNetId] = createData(autGraph,comGraph)

for aNet in autPolNets:
    aNetId = aNet.split("_")[-2]
    if aNetId in names:
        autGraph = nx.read_gpickle(aNet)
        comIdNet = [string for string in comPolNets if aNetId in string]
        comGraph = nx.read_gpickle(comIdNet[0])
        jsonData["t3_"+aNetId] = createData(autGraph,comGraph)

In [35]:
len(jsonData)

684

In [36]:
with open('authorGraphs.json', 'w') as outfile:
    json.dump(jsonData, outfile)

In [54]:
commentsGraph = nx.read_gpickle(net)

In [58]:
commentsGraph.nodes()["t3_m5dkm0"]

{'subset_key': 0,
 'num_comm': 2,
 'created': 1615785898,
 'body': 'Meet Willie! The last of the bunch!',
 'author': 't2_6egedien',
 'score': 30}